# Evaluating Bot Performance (Chess)

In [1]:
import os

parent = '/'.join(s for s in os.getcwd().split('/')[:-1])
print(parent)

/Users/benbellerose/Documents/GitHub/chappie.ai


In [2]:
import pandas as pd

ai = 'test'

game_df = pd.read_csv(f'{parent}/skills/chess/data/models/{ai}/logs/game_log.csv')
game_df.head(10)

,state0,state1,state2,state3,state4,state5,state6,state7,state8,state9,...,prob4092,prob4093,prob4094,prob4095,action,value,reward,ELO,Game-ID,Date
0,0.0,12.0,10.0,11.0,13.0,14.0,11.0,10.0,12.0,9.0,...,0.0,0.0,0.0,0.0,3234.0,1.0,0.0,NaN,1N4DENHJM1ENR0V,2021-12-17 17:45:11.177808
1,1.0,12.0,10.0,11.0,13.0,14.0,11.0,10.0,12.0,9.0,...,0.0,0.0,0.0,0.0,407.0,-1.0,0.0,NaN,1N4DENHJM1ENR0V,2021-12-17 17:45:11.177808
2,0.0,12.0,10.0,11.0,13.0,14.0,11.0,2.0,12.0,9.0,...,0.0,0.0,0.0,0.0,3567.0,1.0,0.0,NaN,1N4DENHJM1ENR0V,2021-12-17 17:45:11.177808
3,1.0,12.0,10.0,11.0,13.0,14.0,11.0,2.0,12.0,9.0,...,0.0,0.0,0.0,0.0,666.0,-1.0,0.0,NaN,1N4DENHJM1ENR0V,2021-12-17 17:45:11.177808
4,0.0,12.0,10.0,11.0,13.0,14.0,11.0,2.0,12.0,9.0,...,0.0,0.0,0.0,0.0,3688.0,1.0,0.0,NaN,1N4DENHJM1ENR0V,2021-12-17 17:45:11.177808
5,1.0,12.0,10.0,11.0,13.0,14.0,11.0,2.0,12.0,9.0,...,0.0,0.0,0.0,0.0,209.0,-1.0,0.0,NaN,1N4DENHJM1ENR0V,2021-12-17 17:45:11.177808
6,0.0,12.0,10.0,11.0,2.0,14.0,11.0,2.0,12.0,9.0,...,0.0,0.0,0.0,0.0,3169.0,1.0,0.0,NaN,1N4DENHJM1ENR0V,2021-12-17 17:45:11.177808
7,1.0,12.0,10.0,11.0,2.0,14.0,11.0,2.0,12.0,9.0,...,0.0,0.0,0.0,0.0,1107.0,-1.0,0.0,NaN,1N4DENHJM1ENR0V,2021-12-17 17:45:11.177808
8,0.0,12.0,10.0,11.0,2.0,14.0,11.0,2.0,12.0,9.0,...,0.0,0.0,0.0,0.0,3641.0,1.0,0.0,NaN,1N4DENHJM1ENR0V,2021-12-17 17:45:11.177808
9,1.0,12.0,10.0,11.0,2.0,14.0,11.0,2.0,12.0,9.0,...,0.0,0.0,0.0,0.0,1262.0,-1.0,0.0,NaN,1N4DENHJM1ENR0V,2021-12-17 17:45:11.177808


In [19]:
import torch

y = 10

for batch, x in enumerate(range(0, len(game_df), y)):
    v_headers = ['value']
    r_headers = ['reward']
    a_headers = ['action']
    s_headers = [h for h in game_df if 'state' in h]
    p_headers = [h for h in game_df if 'prob' in h]
    
    s = game_df[s_headers].iloc[x:x+y]
    p = game_df[p_headers].iloc[x:x+y]
    v = game_df[v_headers].iloc[x:x+y]
    r = game_df[r_headers].iloc[x:x+y]
    a = game_df[a_headers].iloc[x:x+y] + 1
    
    a_0 = pd.DataFrame([{'action':0} for _ in range(len(a))])

    s_1 = game_df[s_headers].shift(periods = -1, axis = 0).iloc[x:x+y]
    if True in s_1.iloc[-1].isna().tolist():
        s_1.iloc[-1] = s.iloc[-1]
        s_1['state0'].iloc[-1] = 0 if s_1['state0'].iloc[-1] == 1 else 1
        
    p_1 = game_df[p_headers].shift(periods = -1, axis = 0).iloc[x:x+y]
    if True in p_1.iloc[-1].isna().tolist():
        p_1.iloc[-1] = p.iloc[-1]
        
    v_1 = game_df[v_headers].shift(periods = -1, axis = 0).iloc[x:x+y]
    if True in v_1.iloc[-1].isna().tolist():
        v_1.iloc[-1] = v.iloc[-1]
        
    r_1 = game_df[r_headers].shift(periods = -1, axis = 0).iloc[x:x+y]
    if True in r_1.iloc[-1].isna().tolist():
        r_1.iloc[-1] = r.iloc[-1]
    
    state = s.append(s, ignore_index = True)
    state = torch.tensor(state.values)
    print('state\n', state, '\n')
    
    a_target = a_0.append(a, ignore_index = True)
    a_target = torch.tensor(a_target.values)
    print('action\n', a_target, '\n')
    
    s_target = s.append(s_1, ignore_index = True)
    s_target = torch.tensor(s_target.values)
    print('state target\n', s_target, '\n')
    
    p_target = p.append(p_1, ignore_index = True)
    p_target = torch.tensor(p_target.values)
    print('policy target\n',p_target, '\n')
    
    v_target = v.append(v_1, ignore_index = True)
    v_target = torch.tensor(v_target.values)
    print('value target\n', v_target, '\n')
    
    r_target = r.append(r_1, ignore_index = True)
    r_target = torch.tensor(r_target.values)
    print('reward target\n', r_target, '\n')
    break

state
 tensor([[ 0., 12., 10.,  ...,  5.,  4.,  6.],
        [ 1., 12., 10.,  ...,  5.,  4.,  6.],
        [ 0., 12., 10.,  ...,  5.,  4.,  6.],
        ...,
        [ 1., 12., 10.,  ...,  5.,  4.,  6.],
        [ 0., 12., 10.,  ...,  5.,  4.,  6.],
        [ 1., 12., 10.,  ...,  5.,  4.,  6.]], dtype=torch.float64) 

action
 tensor([[   0.],
        [   0.],
        [   0.],
        [   0.],
        [   0.],
        [   0.],
        [   0.],
        [   0.],
        [   0.],
        [   0.],
        [3235.],
        [ 408.],
        [3568.],
        [ 667.],
        [3689.],
        [ 210.],
        [3170.],
        [1108.],
        [3642.],
        [1263.]], dtype=torch.float64) 

state target
 tensor([[ 0., 12., 10.,  ...,  5.,  4.,  6.],
        [ 1., 12., 10.,  ...,  5.,  4.,  6.],
        [ 0., 12., 10.,  ...,  5.,  4.,  6.],
        ...,
        [ 0., 12., 10.,  ...,  5.,  4.,  6.],
        [ 1., 12., 10.,  ...,  5.,  4.,  6.],
        [ 0., 12., 10.,  ...,  5.,  4.,  6.]], dtyp

In [16]:
import numpy as np

board = np.zeros((64))
s_headers = [h for h in game_df if 'state' in h]
x_map = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h'] #Board x representation
y_map = ['8', '7', '6', '5', '4', '3', '2', '1'] #Board y representation

for x in range(len(game_df[s_headers]) - 1):
    c = game_df[s_headers].iloc[x]
    n = game_df[s_headers].iloc[x + 1]
    
    #Show current board
    for i in range(len(c)-1):
        board[i] = c.iloc[ i + 1]
    print(board.reshape((8,8)))
    
    #Show next board
    for i in range(len(n)-1):
        board[i] = n.iloc[ i + 1]
    print(board.reshape((8,8)))
    
    #Player action
    b_a = int(game_df['action'].iloc[x])
    a_map = np.zeros(4096)
    a_map[b_a] = 1
    a_map = a_map.reshape((8,8,8,8))
    a_index = [(cy, cx, ny, nx) for cy, cx, ny, nx in zip(*np.where(a_map == 1))][0]
    cur = f'{x_map[a_index[1]]}{y_map[a_index[0]]}'
    next = f'{x_map[a_index[3]]}{y_map[a_index[2]]}'
    print(cur, next)
    print('-----')
    break

[[12. 10. 11. 13. 14. 11. 10. 12.]
 [ 9.  9.  9.  9.  9.  9.  9.  9.]
 [ 2.  2.  2.  2.  2.  2.  2.  2.]
 [ 2.  2.  2.  2.  2.  2.  2.  2.]
 [ 2.  2.  2.  2.  2.  2.  2.  2.]
 [ 2.  2.  2.  2.  2.  2.  2.  2.]
 [ 3.  3.  3.  3.  3.  3.  3.  3.]
 [ 6.  4.  5.  7.  8.  5.  4.  6.]]
[[12. 10. 11. 13. 14. 11. 10. 12.]
 [ 9.  9.  9.  9.  9.  9.  9.  9.]
 [ 2.  2.  2.  2.  2.  2.  2.  2.]
 [ 2.  2.  2.  2.  2.  2.  2.  2.]
 [ 2.  2.  3.  2.  2.  2.  2.  2.]
 [ 2.  2.  2.  2.  2.  2.  2.  2.]
 [ 3.  3.  2.  3.  3.  3.  3.  3.]
 [ 6.  4.  5.  7.  8.  5.  4.  6.]]
c2 c4
-----


In [60]:
p_headers = [h for h in game_df if 'prob' in h]

actions = []
for i, row in game_df[p_headers].iterrows():
    print(row[(row > 0)].to_dict())
    break
    actions.append({k.replace('prob',''):v for k, v in row[(row > 0)].to_dict().items()})

print(pd.DataFrame(actions))

{'prob3104': 0.04, 'prob3112': 0.08, 'prob3169': 0.06, 'prob3177': 0.04, 'prob3234': 0.04, 'prob3242': 0.04, 'prob3299': 0.04, 'prob3307': 0.08, 'prob3364': 0.04, 'prob3372': 0.08, 'prob3429': 0.04, 'prob3437': 0.04, 'prob3494': 0.08, 'prob3502': 0.04, 'prob3559': 0.06, 'prob3567': 0.04, 'prob3688': 0.04, 'prob3690': 0.04, 'prob4013': 0.04, 'prob4015': 0.04}
Empty DataFrame
Columns: []
Index: []


In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import pydot
from networkx.drawing.nx_pydot import graphviz_layout

G = nx.Graph()
for s in search.tree:
    for a in search.tree[s].n_s:
        G.add_edge(s,a)
        
pos = graphviz_layout(G, prog='dot')
#Display network graph -----------------------------
nx.draw(
    G, #Graph nodes & connections
    pos, #Position of graph
    with_labels=True #Labels on nodes
)
plt.rcParams['figure.figsize'] = [40, 40] #Resize graph
plt.show()